# Trajectory Construction 

This note describes how we construct trajectories from photos/movies of [YFCC100M dataset](http://arxiv.org/abs/1502.03409).

**Three major steps for constructing trajectories:**

1. Extract photos/videos taken near Melbourne area from original YFCC100M dataset (`filtering_bigbox.py`)
2. Extract initial trajectories based on the extracted photos/videos (`generate_tables.py`)
3. Filter out some abnormal trajectories using various criteria (in this notebook)


## Construction Process
* [1. Extract initial relevant points from YFCC100M dataset](#1.-Extracting-relevant-points-from-YFCC100M-dataset)
    * [1.1. Basic stats of initial dataset](#1.1.-Basic-stats-of-initial-dataset)
    * [1.2. Scatter plot of extracted points](#1.2.-Scatter-plot-of-extracted-points)
* [2. Extract initial trajectories from extracted points](#2.-Extract-initial-trajectories-from-extracted-points)
    * [2.1 Basic stats after extracting initial trajectories](#2.1-Basic-stats-after-extracting-initial-trajectories)
* [3. Filter Trajectory](#3.-Filter-Trajectory)
    * [3.1. Filter by Timeframe](#3.1.-Filter-by-Timeframe)
        * [3.1.1. Bar chart of taken photos by year](#3.1.1.-Bar-chart-of-taken-photos-by-year)
    * [3.2. Filter by Duration](#3.2.-Filter-by-Duration)
        * [3.2.1. Histogram of trajectory duration](#3.2.1.-Histogram-of-trajectory-duration)
    * [3.3. Filter by Minimum distance](#3.3.-Filter-by-Minimum-distance)
        * [3.3.1. Histogram of trajectory length](#3.3.1.-Histogram-of-trajectory-length)
    * [3.4. Filter by Speed](#3.4.-Filter-by-Speed)
        * [3.4.1. Drop trajectory by average speed](#3.4.1.-Drop-trajectory-by-averag--speed)
            * [3.4.1.1. Histogram of trajectory speed](#3.4.1.1.-Histogram-of-trajectory-speed)
        * [3.4.2. Drop trajectory by point-to-point speed](#3.4.2.-Drop-trajectory-by-point-to-point-speed)
            * [3.4.2.1. Histogram of point-to-point speed](#3.4.2.1.-Histogram-of-point-to-point-speed)
        * [3.4.3. Drop trajectory by sophisticated method](#3.4.3-Drop-trajectory-by-sophisticated-method)
* [4. Filtered Trajectory](#4.-Filtered-Trajectory)
    * [4.1. Basic Stats](#4.1.-Basic-Stats)

# 1. Extract relevant points from YFCC100M dataset

From the original YFCC100M dataset, we first extract the photos/movies belongs to the below region.

![big-box](./img/bigbox.png)

`filtering_bigbox.py` file take the original YFCC100M file to extract photos and videos from above region, and will generate a cvs file containing:
* Photo/video ID
* NSID (user ID)
* Date
* Longitude
* Latitude
* Accuracy (GPS accuracy)
* Photo/video URL
* Photo/video identifier (0 = photo, 1 = video)

The usage of this file is :
> `python filtering_bigbox.py YFCC100M_DATA_FILE`

which will generate `out.YFCC100M_DATA_FILE` file

In [ ]:
%matplotlib inline

import os
import matplotlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import generate_tables
import traj_visualise

data_dir = '../data/'
raw_table = os.path.join(data_dir, 'Melb-bigbox.csv')
raw = pd.read_csv(raw_table, parse_dates=[2], skipinitialspace=True)

## 1.1. Basic stats of initial dataset


In [ ]:
print('Number of photos:', raw['Photo_ID'].shape[0])
print('Number of users: ', raw['User_ID'].unique().shape[0])
raw[['Longitude', 'Latitude', 'Accuracy']].describe() 
# Longitude should be in [141.9, 147.1]
# Latitude should be in [-39.3, -35.8]
# Accuracy should be in [1, 16]

### 1.2. Scatter plot of extracted points

In [ ]:
plt.figure(figsize=[10, 10])
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.scatter(raw['Longitude'], raw['Latitude'])

# 2. Extract initial trajectories from extracted points

With extracted photos(videos) by `filtering_bigbox.py`, we construct initial trajectories with following processes based on several basic criteria.

1. Group photos by user
2. Sort grouped photos by timestamp
3. Split the sorted photos where the second photo of two adjacent photos are taken more than `time_gap` time after
4. Keep trajectories of which at **least one photo** is taken from the below region:

![small-box](./img/smallbox.png)

Here's the argument list:

In [ ]:
extracted_points_file = os.path.join(data_dir, 'Melb-bigbox.csv') # outputfile path of extracted points
time_gap = 8  # hour
minimum_photo = 1 # minimum number of photos for each trajectory

# small bounding box
lng_min = 144.597363
lat_min = -38.072257
lng_max = 145.360413
lat_max = -37.591764

`generate_tables.py` will generate the inital trajectories using above arguments.

In [ ]:
%run generate_tables $extracted_points_file $lng_min $lat_min $lng_max $lat_max $minimum_photo $time_gap 

In [ ]:
table1 = 'Melb-table1.csv'
table2 = 'Melb-table2.csv'
photo_table = os.path.join(data_dir, table1)
traj_table = os.path.join(data_dir, table2)
traj = pd.read_csv(photo_table, parse_dates=[3], skipinitialspace=True)
traj_stats = pd.read_csv(traj_table, parse_dates=[3], skipinitialspace=True)

## 2.1. Basic stats after extracting initial trajectories

In [ ]:
print('Number of photos:', traj['Photo_ID'].shape[0])
print('Number of users: ', traj_stats['User_ID'].unique().shape[0])
print('Number of trajectories:', traj_stats['Trajectory_ID'].shape[0])
print('Average number of photos per user:', \
      1.0 * traj['Photo_ID'].shape[0] / traj_stats['User_ID'].unique().shape[0])
print('Average number of trajectories per user:', \
      1.0 * traj_stats['Trajectory_ID'].shape[0] / traj_stats['User_ID'].unique().shape[0])

In [ ]:
traj_stats[['#Photo', 'Travel_Distance(km)', 'Total_Time(min)', 'Average_Speed(km/h)']].describe()

In [ ]:
traj[['Longitude', 'Latitude', 'Accuracy']].describe()

## 2.2. Scatter plot of points in all trajectories

In [ ]:
plt.figure(figsize=[10, 10])
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.scatter(traj['Longitude'], traj['Latitude'])

## 2.3 Histogram of point-to-point duration

### Histogram of point-to-point duration without time gap cut (i.e. each user with only one long trajectory)

In [ ]:
durations = []
for user in traj_stats['User_ID'].unique():
    photos = traj[traj['User_ID'] == user]
    if photos.shape[0] < 2: continue
    for i in range(photos.index.shape[0]-1):
        p1 = photos.index[i]
        p2 = photos.index[i+1]
        t12 = abs((photos.loc[p1, 'Timestamp'] - photos.loc[p2, 'Timestamp']).total_seconds())
        durations.append(t12 / 60. * 60.) # hour

In [ ]:
pd.Series(durations).describe()

In [ ]:
pd.Series(durations).median()

In [ ]:
plt.figure(figsize=[10, 5])
plt.xlabel('Point-to-Point Time Duration (hour)')
plt.ylabel('#Point-pair')
#ax = pd.Series(durations).hist(bins=5)
#ax.set_yscale('log')
plt.hist(durations, bins=50)
plt.yscale('log', nonposy='clip')

### Histogram of point-to-point duration in all trajectories constructed with time_gap = 24 hours

In [ ]:
durations1 = pd.Series(durations)
durations1 = durations1[durations1 < 24]
#durations1.describe()
pd.DataFrame([durations1.min(), durations1.max(), durations1.mean(), durations1.median()], \
             index=['min', 'max', 'mean', 'median'], \
             columns=['Point-to-Point Duration (hour)'])

In [ ]:
plt.figure(figsize=[10, 5])
plt.xlabel('Point-to-Point Time Duration (hour)')
plt.ylabel('#Point-pair')
#ax = durations1.hist(bins=5)
#ax.set_yscale('log')
plt.hist(durations1.tolist(), bins=20)
plt.yscale('log', nonposy='clip')

### Histogram of point-to-point speed in the trajectory with highest speed

In [ ]:
# filtering by average_speed
maximum_speed = 100 # km/h
traj_stats11 = traj_stats[traj_stats['Average_Speed(km/h)'] < maximum_speed]

In [ ]:
hs_traj_id = traj_stats11.ix[traj_stats11['Average_Speed(km/h)'].idxmax(), 'Trajectory_ID']
hsphotos = traj[traj['Trajectory_ID'] == hs_traj_id]
hspeeds = []
for i in range(hsphotos.index.shape[0]-1):
    p1 = hsphotos.index[i]
    p2 = hsphotos.index[i+1]
    dist = generate_tables.calc_dist(hsphotos.loc[p1, 'Longitude'], hsphotos.loc[p1, 'Latitude'], \
                                     hsphotos.loc[p2, 'Longitude'], hsphotos.loc[p2, 'Latitude'])
    t12 = abs((hsphotos.loc[p1, 'Timestamp'] - hsphotos.loc[p2, 'Timestamp']).total_seconds())
    if t12 == 0: continue
    hspeeds.append(dist * 60. * 60. / t12)

In [ ]:
traj_stats11[traj_stats11['Trajectory_ID'] == hs_traj_id]

In [ ]:
pd.Series(hspeeds).describe()

In [ ]:
pd.Series(hspeeds)

In [ ]:
plt.figure(figsize=[10, 5])
plt.plot(hspeeds, linestyle='dashed', marker='+')
plt.xlabel('Ordered Point-pair')
plt.ylabel('Point-to-Point Speed (km/h)')
#plt.xlabel('Point-to-Point Speed (km/h)')
#plt.ylabel('#Point-pair')
#ax = pd.Series(hspeeds).hist(bins=5)
#ax.set_yscale('log')
#plt.hist(hspeeds, bins=50)
#plt.yscale('log', nonposy='clip')

### 2.4 Histograms of number of photos in trajectories, total time/distance and average speed of trajectories

In [ ]:
plt.figure(figsize=[18, 10])
plt.subplot(2,2,1)
plt.xlabel('#Photo')
plt.ylabel('#Trajectory')
plt.title('Histogram of #Photo in trajectories')
ax0 = traj_stats['#Photo'].hist(bins=50)
ax0.set_yscale('log')

plt.subplot(2,2,2)
plt.xlabel('Travel Distance (km)')
plt.ylabel('#Trajectory')
plt.title('Histogram of Travel Distance of Trajectories')
ax1 = traj_stats['Travel_Distance(km)'].hist(bins=50)
ax1.set_yscale('log')

plt.subplot(2,2,3)
plt.xlabel('Total Time (minute)')
plt.ylabel('#Trajectory')
plt.title('Histogram of Total Time of Trajectories')
ax2 = traj_stats['Total_Time(min)'].hist(bins=50)
ax2.set_yscale('log')

plt.subplot(2,2,4)
plt.xlabel('Average Speed (km/h)')
plt.ylabel('#Trajectory')
plt.title('Histogram of Average Speed of Trajectories')
ax3 = traj_stats['Average_Speed(km/h)'].hist(bins=50)
ax3.set_yscale('log')

# 3. Filter Trajectory

After getting an initial list of trajectories, we further filter out improbable trajectories with various criteria.
We use four different criteria as follows:

1. [Timeframe](#3.1.-Filter-by-Timeframe): We only maintain photos/videos taken during a certain period of time (`start_date`, `end_date`)
2. [Duration](#3.2.-Filter-by-Duration): Some suspicious trajectory span over more than 16 hours. We remove trajectories spanning more than certain minutes (`minimum_duration`).
3. [Minimum distance](#3.3.-Filter-by-Minimum-distance): Trajectories consist of photos taken from single location is not meaningful as a trajectory. We remove these trajectories (`minimum_distance`)
4. [Speed](#3.4-Filter-by-Speed): Due to the GPS error, there are some trajectories in which a user moves unbelievably fast speed. We remove these trajectories, but try to recover as much information as possible from some trajectories.

Here's the list of argument we used to generate final trajectories

In [ ]:
start_date = '2000-01-01'
end_date = '2015-99-99'
minimum_distance = 0.5   # km
speed_filter = 0    #(0: filter by average speed, 1: filter by point-to-point speed, 2: filter by sophispicated method)
maximum_speed = 100    # km/h
maximum_duration = 3000    # minute

## 3.1. Filter by Timeframe

### 3.1.1. Bar chart of taken photos by year

First, we filter out trajectories taken from certain period of time to remove any future photos caused by some errors and too old photos. 

**(The filtering has been done in `filtering_bigbox.py`)**

Here's the bar chart that plots the number of photos taken by each year.

In [ ]:
yeardict = dict()
for i in raw.index:
    dt = raw.ix[i]['Timestamp']
    if dt.year not in yeardict: yeardict[dt.year] = 1
    else: yeardict[dt.year] += 1

In [ ]:
plt.figure(figsize=[9, 5])
plt.xlabel('Year')
plt.ylabel('#Photos')
X = list(sorted(yeardict.keys()))
Y = [yeardict[x] for x in X]
plt.bar(X, Y, width=1)

## 3.2. Filter by Duration

Second, we filter out trajectories which have suspiciously long travel time or the traveling time is zero.

In [ ]:
traj_stats1 = traj_stats[traj_stats['Total_Time(min)'] < maximum_duration]
traj_stats1 = traj_stats1[traj_stats1['Total_Time(min)'] > 0.]
traj1 = traj[traj['Trajectory_ID'].isin(traj_stats1['Trajectory_ID'])]

### 3.2.1. Histogram of trajectory duration

In [ ]:
plt.figure(figsize=[18, 5])
plt.subplot(1,2,1)
plt.xlabel('Travel_Time(min)')
plt.ylabel('#traj')
plt.title('Before Filtering')
ax0 = traj_stats['Total_Time(min)'].hist(bins=50)
ax0.set_yscale('log')

plt.subplot(1,2,2)
plt.xlabel('Travel_Time(min)')
plt.ylabel('#traj')
plt.title('After filtering')
ax1 = traj_stats1['Total_Time(min)'].hist(bins=50)
ax1.set_yscale('log')

##  3.3. Filter by Minimum distance

Third, we filter out trajectories taken from a single location.

In [ ]:
traj_stats2 = traj_stats1[traj_stats1['Travel_Distance(km)'] > minimum_distance]
traj2 = traj[traj['Trajectory_ID'].isin(traj_stats2['Trajectory_ID'])]

#### 3.3.1. Histogram of trajectory length

In [ ]:
plt.figure(figsize=[18, 5])
plt.subplot(1,2,1)
plt.xlabel('Travel_Distance(km)')
plt.ylabel('#traj')
plt.title('Before Filtering')
ax1 = traj_stats1['Travel_Distance(km)'].hist(bins=50)
ax1.set_yscale('log')

plt.subplot(1,2,2)
plt.xlabel('Travel_Distance(km)')
plt.ylabel('#traj')
plt.title('After filtering')
ax2 = traj_stats2['Travel_Distance(km)'].hist(bins=50)
ax2.set_yscale('log')

## 3.4. Filter by Speed

Some trajectories have suspiciously high speed 
There are three (or more) alternative ways to filter out trajectory which has suspiciously high speed.

In [ ]:
traj_stats_new = traj_stats2
traj_new = traj2

### 3.4.1. Drop trajectory by average speed

In [ ]:
if speed_filter == 0:
    traj_stats_new = traj_stats_new[traj_stats_new['Average_Speed(km/h)'] < maximum_speed]
    traj_new = traj_new[traj_new['Trajectory_ID'].isin(traj_stats_new['Trajectory_ID'])]

#### 3.4.1.1. Histogram of trajectory speed

In [ ]:
if speed_filter == 0:
    plt.figure(figsize=[18, 5])
    plt.subplot(1,2,1)
    plt.xlabel('Average_Speed(km/h)')
    plt.ylabel('#traj')
    ax = traj_stats_new['Average_Speed(km/h)'].hist(bins=50)
    ax.set_yscale('log')

### 3.4.2. Drop trajectory by point-to-point speed

There are four cases of supersonic trajectory might be happened

1. The first point of trajectory is far away from the rest of the trajectory (GPS calibrating/entering building etc..)
2. The last point of trajectory is far away from the rest of the trajectory
3. One or more middle points of trajectory are far way from the rest of the trajectory (GPS error)
4. Mixture of previous three cases

The first and second cases are easy to recover by cutting the corresponding point. But it seems we could not easily decide which point(s) should be cut for third and fourth cases. We've decided to remove trajectories in case 3 and 4.


Compute point-to-point speed before filtering

In [ ]:
# compute point-to-point speed
speeds = []
if speed_filter == 1:   
    for tid in traj_stats_new['Trajectory_ID']:
        photos = traj_new[traj_new['Trajectory_ID'] == tid]
        if photos.shape[0] < 2: continue
        for i in range(len(photos.index)-1):
            idx1 = photos.index[i]
            idx2 = photos.index[i+1]
            dist = generate_tables.calc_dist(photos.loc[idx1, 'Longitude'], photos.loc[idx1, 'Latitude'], \
                                             photos.loc[idx2, 'Longitude'], photos.loc[idx2, 'Latitude'])
            seconds = (photos.loc[idx1, 'Timestamp'] - photos.loc[idx2, 'Timestamp']).total_seconds()
            if seconds == 0: continue
            speed = dist * 60. * 60. / abs(seconds)
            speeds.append(speed)

Histogram of point-to-point speed before filtering

In [ ]:
# plot histogram
S = [100, 150, 200, 250, 500, 1000, 1236, 100000]
if speed_filter == 1:
    p2pspeeds = pd.Series(speeds)
    plt.figure(figsize=[18,10])
    for it in range(len(S)):
        plt.subplot(4,2,it+1)
        plt.ylabel('#point-pair')
        plt.title('Speed < ' + str(S[it]) + ' km/h')
        ax = p2pspeeds[p2pspeeds < S[it]].hist(bins=50)
        ax.set_yscale('log')

Drop the first/last point in a trajectories for case1/case2, drop enter trajectories for case3 and case4

In [ ]:
if speed_filter == 1:
    # raise an exception if assigning value to a copy (instead of the original data) of DataFrame
    pd.set_option('mode.chained_assignment','raise') 
    traj_stats_new = traj_stats_new.copy()
    traj_new = traj_new.copy()

    indicator_traj = pd.Series(data=np.ones(traj_stats_new.shape[0], dtype=np.bool), index=traj_stats_new.index)
    indicator_photo = pd.Series(data=np.ones(traj_new.shape[0], dtype=np.bool), index=traj_new.index)
    cnt1 = 0
    cnt2 = 0
    cnt34 = 0
    for i in traj_stats_new['Trajectory_ID'].index:
        tid = traj_stats_new.loc[i, 'Trajectory_ID']
        photos = traj_new[traj_new['Trajectory_ID'] == tid]
        if photos.shape[0] <= 2:
            if traj_stats_new.loc[i, 'Average_Speed(km/h)'] > maximum_speed: # drop the trajectory
                indicator_traj.loc[i] = False
                indicator_photo.loc[photos.index] = False
            continue
        idx1 = photos.index[0]
        idx2 = photos.index[1]
        idx3 = photos.index[-2]
        idx4 = photos.index[-1]
        d12 = generate_tables.calc_dist(photos.loc[idx1, 'Longitude'], photos.loc[idx1, 'Latitude'], \
                                        photos.loc[idx2, 'Longitude'], photos.loc[idx2, 'Latitude'])
        d24 = traj_stats_new.loc[i, 'Travel_Distance(km)'] - d12
        assert(d24 > 0.)
        t12 = abs((photos.loc[idx1, 'Timestamp'] - photos.loc[idx2, 'Timestamp']).total_seconds())
        t24 = abs((photos.loc[idx2, 'Timestamp'] - photos.loc[idx4, 'Timestamp']).total_seconds())
        # check case 1
        if t12 == 0 or (d12 * 60. * 60. / t12) > maximum_speed: #photo1-->photo2, inf speed or large speed
            if t24 == 0 or (d24 * 60. * 60. / t24) > maximum_speed: # drop the trajectory
                indicator_traj.loc[i] = False
                indicator_photo.loc[photos.index] = False
                continue
            else: # case 1, drop the first photo, update trajectory statistics
                #traj_stats.ix[i]['Start_Time'] = photos.ix[idx2]['Timestamp'] # SettingWithCopyWarning
                indicator_photo.loc[idx1] = False
                traj_stats_new.loc[i, 'Start_Time'] = photos.loc[idx2, 'Timestamp']
                traj_stats_new.loc[i, 'Travel_Distance(km)'] = d24
                traj_stats_new.loc[i, 'Total_Time(min)'] = t24 / 60.
                traj_stats_new.loc[i, 'Average_Speed(km/h)'] = d24 * 60. * 60. / t24
                cnt1 += 1
                continue
        # check case 2
        d34 = generate_tables.calc_dist(photos.loc[idx3, 'Longitude'], photos.loc[idx3, 'Latitude'], \
                                        photos.loc[idx4, 'Longitude'], photos.loc[idx4, 'Latitude'])
        d13 = traj_stats_new.loc[i, 'Travel_Distance(km)'] - d34
        t34 = abs((photos.loc[idx3, 'Timestamp'] - photos.loc[idx4, 'Timestamp']).total_seconds())
        t13 = abs((photos.loc[idx1, 'Timestamp'] - photos.loc[idx3, 'Timestamp']).total_seconds())
        if t34 == 0 or (d34 * 60. * 60. / t34) > maximum_speed: #photo3-->photo4, inf speed or large speed
            if t13 == 0 or (d13 * 60. * 60. / t13) > maximum_speed: # drop the trajectory
                indicator_traj.loc[i] = False
                indicator_photo.loc[photos.index] = False
                continue
            else: # case 2, drop the last photo, update trajectory statistics
                #traj_stats.ix[i]['Travel_Distance(km)'] = d13 # SettingWithCopyWarning
                indicator_photo.loc[idx4] = False
                traj_stats_new.loc[i, 'Travel_Distance(km)'] = d13
                traj_stats_new.loc[i, 'Total_Time(min)'] = d13 / 60.
                traj_stats_new.loc[i, 'Average_Speed(km/h)'] = d13 * 60. * 60. / t13
                cnt2 += 1
                continue
            
        # case 3 or 4, drop trajectory
        if traj_stats_new.loc[i, 'Average_Speed(km/h)'] > maximum_speed:
            indicator_traj.loc[i] = False
            indicator_photo.loc[photos.index] = False
            cnt34 += 1
    
    print('Number of trajectories in case 1:', cnt1)
    print('Number of trajectories in case 2:', cnt2)
    print('Number of trajectories in case 3 & 4:', cnt34)

    traj_new = traj_new[indicator_photo]
    traj_stats_new = traj_stats_new[indicator_traj]

Compute point-to-point speed after filtering

In [ ]:
# compute point-to-point speed
speeds_new = []
if speed_filter == 1:
    for tid in traj_stats_new['Trajectory_ID']:
        photos = traj_new[traj_new['Trajectory_ID'] == tid]
        if photos.shape[0] < 2: continue
        for i in range(len(photos.index)-1):
            idx1 = photos.index[i]
            idx2 = photos.index[i+1]
            dist = generate_tables.calc_dist(photos.loc[idx1, 'Longitude'], photos.loc[idx1, 'Latitude'], \
                                             photos.loc[idx2, 'Longitude'], photos.loc[idx2, 'Latitude'])
            seconds = (photos.loc[idx1, 'Timestamp'] - photos.loc[idx2, 'Timestamp']).total_seconds()
            if seconds == 0: continue
            speed = dist * 60. * 60. / abs(seconds)
            speeds_new.append(speed)

### 3.4.3 Drop trajectory by sophisticated method


In [ ]:
#if speed_filter == 2:

# 4. Filtered Trajectory

## 4.1. Basic Stats

More detail analysis will be included in `filckr_analysis.ipynb` and slides. Here we show simple stats from the final result.

In [ ]:
traj_stats_new.describe()

**Save final trajectories to the data folder**

In [ ]:
file1 = os.path.join(data_dir + table1)
file2 = os.path.join(data_dir + table2)
traj_new.to_csv(file1, index=False)
traj_stats_new.to_csv(file2, index=False)